# The Simpsons
![The Simpsons](https://upload.wikimedia.org/wikipedia/ru/4/42/Simpsons_cast.jpg)

...one more pipeline...

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as tfs
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load data

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     print('{} CONTAINS {} files'.format(dirname, len(filenames)))

In [ ]:
input_dir = '/kaggle/input/journey-springfield/train/simpsons_dataset/'
rescale_size = 244
batch_size = 32 # 32 - resnet152. 64 - all another
# Imagenet mean and standard (are calculated from all of images)
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
transform = tfs.Compose([
    tfs.Resize((int(rescale_size * 1.25), int(rescale_size * 1.25))),
    tfs.RandomCrop(rescale_size),
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),
    tfs.Normalize(imagenet_mean, imagenet_std)
])

dataset_full = torchvision.datasets.ImageFolder(root=input_dir, transform=transform)

# split full dataset
train_idx, valid_idx = train_test_split(list(range(len(dataset_full))), train_size=0.8)
dataset = {
    'train': torch.utils.data.Subset(dataset_full, train_idx),
    'valid': torch.utils.data.Subset(dataset_full, valid_idx)
}

dataset_size = {ds: len(dataset[ds]) for ds in ['train', 'valid']}
dataset_classes = np.array(dataset_full.classes)
print('classes:', dataset_classes, '\nsize', dataset_size)

dataloader = {
    'train': torch.utils.data.DataLoader(
        dataset=dataset['train'], batch_size=batch_size, shuffle=True, num_workers=2
    ),
    'valid': torch.utils.data.DataLoader(
        dataset=dataset['valid'], batch_size=batch_size, shuffle=False, num_workers=2
    ),
}

In [ ]:
transform = tfs.Compose([
    tfs.Resize((int(rescale_size * 1.05), int(rescale_size * 1.05))),
    tfs.CenterCrop(rescale_size),
    tfs.ToTensor(),
    tfs.Normalize(imagenet_mean, imagenet_std)
])
dataset_test = torchvision.datasets.ImageFolder(
    root='/kaggle/input/journey-springfield/testset',
    transform=transform
)
dataloader_test = torch.utils.data.DataLoader(dataset_test, shuffle=False, batch_size=batch_size)
test_filenames = [fn[0].split('/')[-1] for fn in dataset_test.imgs]

# Show some data

In [ ]:
def get_batches(dataloader, num_of_batches=1):
    assert(num_of_batches > 0)
    images, classes = next(iter(dataloader))
    for _ in range(num_of_batches - 1):
        image_batch, cls_batch = next(iter(dataloader))
        images = torch.cat([images, image_batch], dim=0)
        classes = torch.cat([classes, cls_batch], dim=0)
    return images, classes

In [ ]:
def show_batches(images, classes):
    imagenet_mean = np.array([0.485, 0.456, 0.406])
    imagenet_std = np.array([0.229, 0.224, 0.225])
    cols = (images.shape[0] + 3) // 4
    images = images.numpy().transpose(0,2,3,1)
    images = images * imagenet_std + imagenet_mean
    fig, axes = plt.subplots(cols, 4, figsize=(12, cols * 3))
    for ax, img, img_cls in zip(axes.flatten(), images, classes.numpy()):
        ax.imshow(img)
        ax.set_xticks([])
        ax.set_xlabel(dataset_classes[img_cls], fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# images, classes = get_batches(dataloader['train'])
# show_batches(images, classes)

# Fit functions

In [ ]:
def train_model(
    model: torch.nn.Module,
    criterion: torch.nn.Module,
    optimizer: torch.nn.Module,
    sheduler: torch.nn.Module,
    epochs: int = 10
) -> (torch.nn.Module, dict):
    r"""Training the model. Returns best model, dictionary of train and validation losses, dictionary of train and validation accuracies.
    Args:
        model (torch.nn.Module): Neural network
        criterion (torch.nn.Module): Cost function
        optimizer (torch.nn.Module): Optimization algorithm
        sheduler (torch.nn.Module): Learning rate change policy
        epochs (int): Number of training iterations. Default: 25
    """
    
    losses = {'train': [], 'valid': []}
    accuracies = {'train': [], 'valid': []}
    best_model_weights = model.state_dict()
    best_accuracy = 0.
    progress = tqdm(range(epochs), desc='Epoch:')
    
    for epoch in progress:
        for phase in ['train', 'valid']:
            loss_accum = 0.
            corr_accum = 0
            model.train(mode=(phase == 'train'))
            
            for inputs, labels in tqdm(dataloader[phase], desc=f'Phase {phase}:'):
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
            
                if phase == 'train':
                    optimizer.zero_grad()
                    outp = model(inputs)
                    loss = criterion(outp, labels)
                    loss.backward()
                    optimizer.step()
                else:
                    with torch.no_grad():
                        outp = model(inputs)
                        loss = criterion(outp, labels)
                
                preds = torch.argmax(outp, -1)
                loss_accum += loss.item()
                corr_accum += (preds == labels.data).sum()

            if phase == 'train':
                sheduler.step()
                
            epoch_loss = loss_accum / dataset_size[phase]
            epoch_accuracy = corr_accum / dataset_size[phase]
            losses[phase].append(epoch_loss)
            accuracies[phase].append(epoch_accuracy)
            progress.set_description('loss: {:.4f}, acc: {:.4f}'.format(epoch_loss, epoch_accuracy))
            if phase == 'valid' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                best_model_weights = model.state_dict()
    
    model.load_state_dict(best_model_weights)
    return model, losses, accuracies

In [ ]:
def show_metrics(metrics):
    plt.figure(figsize=(12, 8))
    plt.plot(metrics['train'], label='train')
    plt.plot(metrics['valid'], label='val')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
def show_metrics_all(losses, accuracies):
    def show_ax(ax_idx, metric, title):
        axes[ax_idx].plot(metric['train'], label='train')
        axes[ax_idx].plot(metric['valid'], label='valid')
        axes[ax_idx].grid()
        axes[ax_idx].legend()
        axes[ax_idx].set_title(title)

    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    show_ax(0, losses, 'loss')
    show_ax(1, accuracies, 'accuracy')
    plt.show()

# Predict functions

In [ ]:
def predict(model, dataloader_test):
    logits = []
    model.eval()
    with torch.no_grad():
        for inputs, _ in tqdm(dataloader_test):
            inputs = inputs.to(DEVICE)
            outputs = model(inputs).cpu()
            logits.append(outputs)
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
def predict_to_submission(model, dataloader_test=dataloader_test, test_filenames=test_filenames, dataset_classes=dataset_classes):
    probs = predict(model, dataloader_test)
    print(probs.shape)
    predictions = np.argmax(probs, axis=1)
    preds = dataset_classes[predictions]
    submission = pd.read_csv('/kaggle/input/journey-springfield/sample_submission.csv')
    submission = pd.DataFrame({'Id': test_filenames, 'Expected': preds}).sort_values('Id')
    submission.to_csv('./submission.csv', index=False)

# Simple Model

In [ ]:
class SimpleCNN(nn.Module):
    
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=96, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.out = nn.Linear(96 * 5 * 5, n_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), -1)
        logits = self.out(x)
        return logits

In [ ]:
model_scnn = SimpleCNN(len(dataset_full.classes))
model

In [ ]:
model.to(DEVICE)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
sheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model, losses, accuracies = train_model(model, loss, optimizer, sheduler)
show_metrics_all(losses, accuracies)
predict_to_submission(model)

# AlexNet

In [ ]:
# model_alexnet = torchvision.models.alexnet(pretrained=True)

In [ ]:
# for param in model_alexnet.features.parameters():
#     param.requires_grad = False
# num_in_features = 4096
# num_out_features = len(dataset_full.classes)
# model_alexnet.classifier[6] = nn.Linear(in_features=num_in_features, out_features=num_out_features, bias=True)
# # for param in model_alexnet.parameters():
# #     print(param.requires_grad)
# model_alexnet

In [ ]:
# model_alexnet.to(DEVICE)
# loss = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model_alexnet.parameters(), lr=1e-3)
# sheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# model_alexnet, losses, accuracies = train_model(model_alexnet, loss, optimizer, sheduler)

In [ ]:
# show_metrics_all(losses, accuracies)

In [ ]:
# predict_to_submission(model_alexnet)